In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('/home/vyacheslav/Data Scientist/IV ЧЕТВЕРТЬ/II. Рекомендательные системы/2 lesson/data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [6]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

   Ответ:,
   1. 999999 - фиктивный товар;
   2. Этот товар нужен для того, если юзер НЕ покупал товары из топ-5000, то он фиктивный товар;
   3. Этот товар среди топ добавляется вперед, т.е. данный товар будет покупать каждый юзер. Значит качество рекомендаций смещается вправо;
   4. Можно, но тогда если юзер ничего не купил, то данный юзер для нас не будет нести никакой информации.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [8]:
items = data_train.item_id.unique()

sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
weights = sales['sales_value'].apply(lambda x: np.log(x) if x > 0 else 0).values

weights = np.array([x if x >= 0 else 0 for x in weights])

total_sum = weights.sum()
weights = weights / total_sum

In [9]:
def weighted_random_recommendation(items, weights, n=5):
    """Взвешенные случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [12]:
result = pd.read_csv('preds.csv')# закгрузка predict с семианара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2996422, 1001662, 1115785, 10182678, 8019186]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9531136, 12987938, 959773, 966619, 15828252]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


Сделайте предсказания

In [13]:
%%time

# your_code

items = data_train.item_id.unique()

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))
result.head(2)

CPU times: user 2.58 s, sys: 0 ns, total: 2.58 s
Wall time: 2.58 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2996422, 1001662, 1115785, 10182678, 8019186]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[16829145, 80132, 1006702, 1033887, 1239506]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9531136, 12987938, 959773, 966619, 15828252]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[6396402, 827379, 930870, 1107477, 820660]"


### Задание 2. Расчет метрик

Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [58]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [59]:
def func_precision(method: str):
    k = 5
    return result.apply(lambda x: precision_at_k(x[method], x['actual'],  k), axis=1).mean()

In [60]:
random = func_precision('random_recommendation')
popular = func_precision('popular_recommendation')
itemitem = func_precision('itemitem')
cosine = func_precision('cosine')
tfidf = func_precision('tfidf')
own_purchases = func_precision('own_purchases')
weight_random = func_precision('weighted_random_recommendation')


print('pr@5 random:', '%.3f' % random)
print('pr@5 popular:', '%.3f' % popular)
print('pr@5 itemitem:', '%.3f' % itemitem)
print('pr@5 cosine:', '%.3f' % cosine)
print('pr@5 tfidf:', '%.3f' % tfidf)
print('pr@5 own_purchases:', '%.3f' % own_purchases)
print('pr@5 weight_random:', '%.3f' % weight_random)

pr@5 random: 0.000
pr@5 popular: 0.155
pr@5 itemitem: 0.137
pr@5 cosine: 0.133
pr@5 tfidf: 0.139
pr@5 own_purchases: 0.180
pr@5 weight_random: 0.000


Вывод: лучшее качество показывает own_purchases, т.к. не учитывается введенный ранее товар 999999.

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [62]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [63]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [71]:
sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()[:5000]
weights = sales['sales_value'].apply(lambda x: np.log(x) if x > 0 else 0).values

weights = np.array([x if x >= 0 else 0 for x in weights])

топ-5000

In [72]:
result['random_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result['weight_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(top_5000, weights, n=5))

result.head(2)

ValueError: probabilities do not sum to 1

Дальше не смог сделать

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.